# AthenaK scaling on ALCF Polaris

2022-05-19

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np

import pandas as pd

mpl.rcParams['figure.dpi'] = 300

from IPython.display import Image

import seaborn as sns


## Strong scaling, 1 to 16 nodes

In [ ]:
data = pd.read_csv('strong_scaling.csv')
data['Speedup'] = data['zone-cycles/cpu_second']/data['zone-cycles/cpu_second'][0]

In [ ]:
data

In [ ]:
data['zone-cycles/cpu_second'][0]

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['zone-cycles/cpu_second'],'-o')
#ax.set_xlabel('Number of MPI ranks = N_A100')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')

# ax.set_xscale('log', base=2)

ax.set_ylabel('Zone-cycles/second')
ax.axvline(x=4, color='0.8', alpha=0.8)


In [ ]:
fig.savefig("strong-scaling.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['Speedup'],'-o')
#ax.set_xlabel('Number of MPI ranks = N_A100')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Speedup')

ax.axvline(x=4, color='0.8', alpha=0.8)


In [ ]:
fig.savefig("strong-scaling-speedup.pdf")

## Weak scaling